### Dynamic Programming Notes

Dynamic Programming is mainly an optimization over plain recursion. Wherever we see a recursive solution that has repeated calls for same inputs, we can optimize it using Dynamic Programming. The idea is to simply store the results of subproblems, so that we do not have to re-comupute them when needed later. This simple optimization reduces time complexities from exponential to polynomial.

For example, if we write simple recursive solution for Fibonacci Numbers, we get exponential time complexity and if we optimize it by storing solutions of subproblems, time complexity reduces to linear.

In [2]:
import functools
import time

In [41]:
def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args,**kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

In [7]:
def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args,**kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

In [6]:
def cache(func):
    """Keep a cache of previous function calls"""
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = dict()
    return wrapper_cache

In [34]:
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)
start_time = time.perf_counter()
fibonacci(35)
end_time = time.perf_counter()
run_time = end_time - start_time
print('Total Runtime is: {} sec.'.format(str(run_time)))

Total Runtime is: 4.691171081998618 sec.


In [35]:
@cache
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)
start_time = time.perf_counter()
fibonacci(35)
end_time = time.perf_counter()
run_time = end_time - start_time
print('Total Runtime is: {} sec.'.format(str(run_time)))

Total Runtime is: 0.00023535999935120344 sec.


In [36]:
@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)
fibonacci(5)

Call 1 of 'fibonacci'
Call 2 of 'fibonacci'
Call 3 of 'fibonacci'
Call 4 of 'fibonacci'
Call 5 of 'fibonacci'
Call 6 of 'fibonacci'
Call 7 of 'fibonacci'
Call 8 of 'fibonacci'
Call 9 of 'fibonacci'
Call 10 of 'fibonacci'
Call 11 of 'fibonacci'
Call 12 of 'fibonacci'
Call 13 of 'fibonacci'
Call 14 of 'fibonacci'
Call 15 of 'fibonacci'


5

In [37]:
@cache
@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)
fibonacci(5)

Call 1 of 'fibonacci'
Call 2 of 'fibonacci'
Call 3 of 'fibonacci'
Call 4 of 'fibonacci'
Call 5 of 'fibonacci'
Call 6 of 'fibonacci'


5

Example 1: Longest Common Subsequence

Definition 11.1 The Longest Common Subsequence (LCS) problem is as follows. We are
given two strings: string S of length n, and string T of length m. Our goal is to produce their longest common subsequence: the longest sequence of characters that appear left-to-right (but not necessarily in a contiguous block) in both strings

LCS Problem Statement: Given two sequences, find the length of longest subsequence present in both of them. A subsequence is a sequence that appears in the same relative order, but not necessarily contiguous. For example, “abc”, “abg”, “bdf”, “aeg”, ‘”acefg”, .. etc are subsequences of “abcdefg”. So a string of length n has 2^n different possible subsequences.

It is a classic computer science problem, the basis of diff (a file comparison program that outputs the differences between two files), and has applications in bioinformatics.

Examples:

LCS for input Sequences “ABCDGH” and “AEDFHR” is “ADH” of length 3.

LCS for input Sequences “AGGTAB” and “GXTXAYB” is “GTAB” of length 4.

In [58]:
# pure recursive
def lcs(X, Y, m, n):   
    if m == 0 or n == 0: 
        return 0; 
    elif X[m-1] == Y[n-1]: 
        return 1 + lcs(X, Y, m-1, n-1); 
    else: 
        return max(lcs(X, Y, m, n-1), lcs(X, Y, m-1, n)); 
  
# Driver program to test the above function 
X = "AGGTAB"
Y = "GXTXAYB"
print("Length of LCS is: ", lcs(X , Y, len(X), len(Y)))

Length of LCS is:  4


Time complexity of the above naive recursive approach is O(2^n) in worst case and worst case happens when all characters of X and Y mismatch i.e., length of LCS is 0.

                 lcs("AXYT", "AYZX")
                       /                 
         lcs("AXY", "AYZX")                        lcs("AXYT", "AYZ")
         /                                         /               
     lcs("AX", "AYZX") lcs("AXY", "AYZ")        lcs("AXY", "AYZ") lcs("AXYT", "AY")

In [56]:
# method 1:
def lcs(X , Y): 
    # find the length of the strings 
    m = len(X) 
    n = len(Y) 
  
    # declaring the array for storing the dp values 
    L = [[None]*(n+1) for i in range(m+1)] 
  
    """Following steps build L[m+1][n+1] in bottom up fashion 
    Note: L[i][j] contains length of LCS of X[0..i-1] 
    and Y[0..j-1]"""
    for i in range(m+1): 
        for j in range(n+1): 
            if i == 0 or j == 0 : 
                L[i][j] = 0
            elif X[i-1] == Y[j-1]: 
                L[i][j] = L[i-1][j-1]+1
            else: 
                L[i][j] = max(L[i-1][j] , L[i][j-1]) 
  
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1] 
    return L[m][n]

X = "AGGTAB"
Y = "GXTXAYB"
print("Length of LCS is: ", lcs(X, Y))

Length of LCS is:  4


In [57]:
# method 2:
@cache
@count_calls
def lcs(X, Y, m, n):   
    if m == 0 or n == 0: 
        return 0; 
    elif X[m-1] == Y[n-1]: 
        return 1 + lcs(X, Y, m-1, n-1); 
    else: 
        return max(lcs(X, Y, m, n-1), lcs(X, Y, m-1, n)); 
  
X = "AGGTAB"
Y = "GXTXAYB"
print("Length of LCS is: ", lcs(X , Y, len(X), len(Y)))

Call 1 of 'lcs'
Call 2 of 'lcs'
Call 3 of 'lcs'
Call 4 of 'lcs'
Call 5 of 'lcs'
Call 6 of 'lcs'
Call 7 of 'lcs'
Call 8 of 'lcs'
Call 9 of 'lcs'
Call 10 of 'lcs'
Call 11 of 'lcs'
Call 12 of 'lcs'
Call 13 of 'lcs'
Call 14 of 'lcs'
Call 15 of 'lcs'
Call 16 of 'lcs'
Call 17 of 'lcs'
Call 18 of 'lcs'
Call 19 of 'lcs'
Call 20 of 'lcs'
Call 21 of 'lcs'
Call 22 of 'lcs'
Call 23 of 'lcs'
Call 24 of 'lcs'
Call 25 of 'lcs'
Call 26 of 'lcs'
Call 27 of 'lcs'
Call 28 of 'lcs'
Call 29 of 'lcs'
Call 30 of 'lcs'
Call 31 of 'lcs'
Call 32 of 'lcs'
Length of LCS is:  4


221. Maximal Square

Given a 2D binary matrix filled with 0's and 1's, find the largest square containing only 1's and return its area.

Example:

Input: 

1 0 1 0 0

1 0 1 1 1

1 1 1 1 1

1 0 0 1 0

Output: 4

link:https://www.youtube.com/watch?v=vkFUB--OYy0